## Youtube 스크립트 텍스트 전처리

In [41]:
import pandas as pd
import numpy as np
from hanspell import spell_checker # 500글자 이상의 문장은 검사 불가
import re
import warnings 
warnings.filterwarnings('ignore')

from konlpy.tag import Okt
okt = Okt()

In [2]:
path = input('절대 경로 입력 : ')

절대 경로 입력 :  C:\Users\j\stock_predict\Sentiment_Analysis\data\전처리 전 데이터\df_youtube_삼프로LG화학.csv


In [110]:
df1 = pd.read_csv(path)

In [111]:
df1.text

0      으[음악]tera term[음악]으 으 으 으 으[음악]이름은 하나인데 직업은 서먹...
1      구독과 좋아요는 사람입니다자 염치 염치 영차 차 우리 염성 안 차장에 보시겠습니다 ...
2      좋아요 구독 눌러주세요자 그럼 오늘 첫 번째 뉴스 부터 시작을 해 봐야죠먼저 뭐 미...
3      자 그러면 오늘 노려 일이니까요 발칙하게 한번 가보겠습니다뭐 리토의 방송에 건수를 ...
4      산업을 이해하고 기업을 이해해야 돈의 흐름이 보입니다리서치 명과 메리츠 증권과 3%...
                             ...                        
502    자 그러면 시간이 달했습니다 우리염불 이염 20 이상이면 경우 아침개장 상황 살펴보...
503    자 그러면 오늘 금요일 아침 9월10일 아침 증시 개장 은 어떻게 될것인지 우리 염...
504    4자 오늘 9월 9일 목요일 전혀아유진 자산운용의 김 타기 산이 모시고실제 에 돈을...
505    자 오늘 저희가 함께 이야기 나눌 분은 요 예 gs 자산운용예전에 그런 노래가 하나...
506    으[음악]으 으[음악]on으[박수][음악]기피하는 경제의 컨텐츠 오나 10나이브 져...
Name: text, Length: 507, dtype: object

In [112]:
# 데이터 전처리

df1.text = df1.text.str.replace('[으*]','')
df1.text = df1.text.str.replace('[\[\]]','')
df1.text = df1.text.str.replace('음악','')
df1.text = df1.text.str.replace('자','')
df1.text = df1.text.str.replace('[a-zA-Z]','')
df1.text = df1.text.str.replace('[구독*]','')
df1.text = df1.text.str.replace('[좋아요]','')
df1.text = df1.text.str.replace('[\s]','')
df1.text = df1.text.str.strip() # 앞뒤 공백 제거

df1.text 

0      이름은하나인데직업은서먹이라는말들어봤어화직업이소노계란이크게가능해본절본섬불운박극혐부부지...
1      과는사람입니다염치염치영차차우리염성안차장에보시겠습니다어서오십시오어서오세예어제밤새리포트...
2      눌러주세그럼오늘첫번째뉴스부터시작을해봐야죠먼저뭐미연준얘기근데뭐당연히매실얘기를먼저해야할...
3      그러면오늘노려일이니까발칙하게한번가보겠습니다뭐리토의방송에건수를기님어서오십시오이안녕하세...
4      산업을이해하고기업을이해해야돈의흐름이보입니다리서치명과메리츠증권과3%가함께야심차게준비한...
                             ...                        
502    그러면시간이달했습니다우리염불이염20이상이면경우침개장상황살펴보겠습니다베스트투증권의염승...
503    그러면오늘금일침9월10일침증시개장은어떻게될것인지우리염불베스트투증권의염승환이사님과함께...
504    4오늘9월9일목일전혀유진산운용의김타기산이모시고실제에돈을본편이좀그런가에는굴리는문들은어...
505    오늘저희가함께이야기나눌분은예산운용예전에그런노래가하나있었죠성은김이름은뒤했어그리고누가불...
506    박수기피하는경제의컨텐츠오나10나이브져용지힘도원입니다4장4예언하신라이브9월12일금일입...
Name: text, Length: 507, dtype: object

In [113]:
# 불용어
url = 'https://raw.githubusercontent.com/chaerui7967/stock_predict_news_and_youtube/master/Sentiment_Analysis/data/stopwords_ver1.txt'
stopwords = list(pd.read_csv(url, header=None)[0])

In [114]:
df1['token'] = ''

In [115]:
# test 10개만 빠르게 수행
df2 = df1[:10]

In [116]:
# 토큰화
for i in range(len(df2)):
    temp = []
    temp = okt.nouns(df2['text'].iloc[i])
    temp = [word for word in temp if not word in stopwords] # 불용어 제거
    temp = [word for word in temp if len(word) > 1] # 불용어 제거
    print(i,'번 토큰화, 불용어 제거 완료')
    
    temp = list(set(temp))
    print(i, ' 중복단어 제거')
    
    temp = [spell_checker.check(word).checked for word in temp]
    print(i,'- spell check')
    
    df2['token'].iloc[i] = temp
    print(i,'- 완료')

0 번 토큰화, 불용어 제거 완료
0  중복단어 제거
0 - spell check
0 - 완료
1 번 토큰화, 불용어 제거 완료
1  중복단어 제거
1 - spell check
1 - 완료
2 번 토큰화, 불용어 제거 완료
2  중복단어 제거
2 - spell check
2 - 완료
3 번 토큰화, 불용어 제거 완료
3  중복단어 제거
3 - spell check
3 - 완료
4 번 토큰화, 불용어 제거 완료
4  중복단어 제거
4 - spell check
4 - 완료
5 번 토큰화, 불용어 제거 완료
5  중복단어 제거
5 - spell check
5 - 완료
6 번 토큰화, 불용어 제거 완료
6  중복단어 제거
6 - spell check
6 - 완료
7 번 토큰화, 불용어 제거 완료
7  중복단어 제거
7 - spell check
7 - 완료
8 번 토큰화, 불용어 제거 완료
8  중복단어 제거
8 - spell check
8 - 완료
9 번 토큰화, 불용어 제거 완료
9  중복단어 제거
9 - spell check
9 - 완료


In [117]:
df2['token']

0    [비상, 주가, 생도, 집단, 피카, 도안, 과속, 야만, 에디션, 발표, 이만, ...
1    [비상, 제안, 주가, 이태, 조성, 이사회, 건가, 을해, 반응, 면지, 반사, ...
2    [주가, 건가, 이사회, 을해, 리가, 언어, 장사, 통화정책, 가야, 짜리, 도우...
3    [물어, 은폐, 건가, 을해, 리가, 게리, 리나, 중고차, 안전성, 노력, 어려움...
4    [주가, 제안, 출발, 을해, 리가, 제조업, 금융, 가야, 중생, 증권사, 금오,...
5    [주가, 서일본, 안성, 을해, 부락, 우의, 가야, 로비, 우이, 진작, 에어, ...
6    [주가, 출발, 을해, 사망, 이파리, 장사, 금융, 증권사, 양물, 을 것, 날짜...
7    [주가, 제안, 비상, 상상, 을해, 외출, 별미, 언어, 금융, 증권사, 노력, ...
8    [출발, 을해, 반응, 언어, 금융, 차장, 증권사, 힐링, 팬덤, 현금, 신속, ...
9    [주가, 출발, 을해, 대표이사, 정원, 언어, 차장, 유엔, 증권사, 리나, 도안...
Name: token, dtype: object

In [ ]:
import codecs

positive = [] 
negative = []
posneg = []
dic = [('./KOSELF_pos.txt', './KOSELF_neg.txt'),
       ("./data/positive_words_self.txt","./data/negative_words_self.txt"),
      ('./data/sum_pos.txt','./data/sum_neg.txt')]

i = int(input('koself : 1, blog : 2, sum : 3'))-1


pos = codecs.open(dic[i][0], 'rb', encoding='UTF-8') 

while True: 
    line = pos.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
        
    positive.append(line) 
    posneg.append(line) 
        
pos.close() 

neg = codecs.open(dic[i][1], 'rb', encoding='UTF-8') 
while True: 
    line = neg.readline() 
    line = line.replace('\n', '')
    line = line.replace('\r', '')
    
    if not line: break
    
    negative.append(line) 
    posneg.append(line) 
    
neg.close()